## Tire Strategy Example

In [41]:
import fastf1
import fastf1.plotting
from matplotlib import pyplot as plt
import pandas as pd

pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)
pd.set_option('max_colwidth', 400)

In [13]:
# Create a session to load data for.
session = fastf1.get_session(2022, 'Silverstone', 'R')

# Load the session data.
session.load()

core           INFO 	Loading data for British Grand Prix - Race [v3.1.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

In [38]:
# Get the laps completed by drivers.
laps = session.laps

In [39]:
# List of driver numbers.
print(session.drivers)

['55', '11', '44', '16', '14', '4', '1', '47', '5', '20', '18', '6', '3', '22', '31', '10', '77', '63', '24', '23']


In [44]:
# You can pass the numbers into a function to produce their three letter code.
driver_codes = [session.get_driver(driver)['Abbreviation'] for driver in session.drivers]
print(driver_codes)

['SAI', 'PER', 'HAM', 'LEC', 'ALO', 'NOR', 'VER', 'MSC', 'VET', 'MAG', 'STR', 'LAT', 'RIC', 'TSU', 'OCO', 'GAS', 'BOT', 'RUS', 'ZHO', 'ALB']


In [46]:
# Need to find the stint length and compound used for every stint by every driver.

#laps.groupby(['Driver', 'Stint'])['Compound'].apply(set)

df = laps[['Driver'],]

Driver  Stint
ALB     1.0        {SOFT}
ALO     1.0      {MEDIUM}
        2.0      {MEDIUM}
        3.0        {HARD}
        4.0        {SOFT}
BOT     1.0      {MEDIUM}
        2.0      {MEDIUM}
GAS     1.0      {MEDIUM}
        2.0        {SOFT}
        3.0        {HARD}
HAM     1.0      {MEDIUM}
        2.0      {MEDIUM}
        3.0        {HARD}
        4.0        {SOFT}
LAT     1.0        {SOFT}
        2.0        {SOFT}
        3.0      {MEDIUM}
        4.0        {SOFT}
LEC     1.0      {MEDIUM}
        2.0      {MEDIUM}
        3.0        {HARD}
MAG     1.0        {SOFT}
        2.0      {MEDIUM}
        3.0      {MEDIUM}
MSC     1.0      {MEDIUM}
        2.0      {MEDIUM}
        3.0        {HARD}
        4.0        {SOFT}
NOR     1.0      {MEDIUM}
        2.0      {MEDIUM}
        3.0        {HARD}
        4.0        {SOFT}
OCO     1.0      {MEDIUM}
        2.0        {SOFT}
        3.0        {HARD}
PER     1.0      {MEDIUM}
        2.0      {MEDIUM}
        3.0      {MEDIUM

In [ ]:




###############################################################################
# We need to find the stint length and compound used
# for every stint by every driver.
# We do this by first grouping the laps by the driver,
# the stint number, and the compound.
# And then counting the number of laps in each group.
stints = laps[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"])
stints = stints.count().reset_index()

###############################################################################
# The number in the LapNumber column now stands for the number of observations
# in that group aka the stint length.
stints = stints.rename(columns={"LapNumber": "StintLength"})
print(stints)

###############################################################################
# Now we can plot the strategies for each driver
fig, ax = plt.subplots(figsize=(5, 10))

for driver in drivers:
    driver_stints = stints.loc[stints["Driver"] == driver]

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        # each row contains the compound name and stint length
        # we can use these information to draw horizontal bars
        plt.barh(
            y=driver,
            width=row["StintLength"],
            left=previous_stint_end,
            color=fastf1.plotting.COMPOUND_COLORS[row["Compound"]],
            edgecolor="black",
            fill=True
        )

        previous_stint_end += row["StintLength"]

# sphinx_gallery_defer_figures

###############################################################################
# Make the plot more readable and intuitive
plt.title("2022 Hungarian Grand Prix Strategies")
plt.xlabel("Lap Number")
plt.grid(False)
# invert the y-axis so drivers that finish higher are closer to the top
ax.invert_yaxis()

# sphinx_gallery_defer_figures

###############################################################################
# Plot aesthetics
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()